[Backround](https://tech.instacart.com/3-million-instacart-orders-open-sourced-d40d29ead6f2)

TBD:  
* What is the skipping thing?
* Should we throw out reordering=1 during initial read?
* Need to normalize numbers. Should keep track of mean and min/max during input
* with ```open("data/products.csv",  "r", encoding='utf8') as f_obj:```
* for now we ignore products in an order that were not reordered
* need to add reorder number to the data as a feature. reorder number and order num are need to produce the average.
* the size of the neural net should grow as the amount of data about the person grows.
* should try and predict the number of orders
    

The idea here is to create data with expanded orders of individual items.
Each item ordered will then have all the information about it, including the order it came from and all other information.

The label will be if it is reordered.

The network will be solved for each person individually based on their order history.
I don't think that other peopls history will be a good indicator as to what this person ordered, unless they are already doing a good job of suggesting products they might want to buy.

The names of columns will be:
* product_id
* aisle_id
* department_id
* add_to_cart_order (not used for now)
* order_id  (global order id)
* user_id (not needed if just specific to user)
* order_num (order number of that user)
* order_dow
* order_hour_of_day
* days_since_prior_order  (not sure if NAN is the right mathematical value for first order.

maybe infinity but will have to experiment. possibly i don't
want to enter any past data for a non repeat order; since we
know about the last time it was ordered.)
                         

The label is:
* reordered

Table order is an actual product order.

Orders in an aggregate, i.e. one or more items being ordered at the same time.

We will test these one at a time. For every set associated with an orders element, we will
create all the ones for products that person has every ordered and make it fail, i.e.
the label will be false, during training.

We don't really care about aisles and departments, those are just
the ascii names for the ids.


In [ ]:
import pandas as pd
import numpy as np
import time

In [ ]:
start = time.time()
end = time.time()
print('{0} seconds elapsed time '.format(end - start))

In [ ]:
start = time.time()
aisles =      pd.read_csv('data/aisles.csv')
departments = pd.read_csv('data/departments.csv')
order_train = pd.read_csv('data/order_products__train.csv')
order_prior = pd.read_csv('data/order_products__prior.csv')
products =    pd.read_csv('data/products.csv')
orders =      pd.read_csv('data/orders.csv')
end = time.time()
print('{0} seconds elapsed time '.format(end - start))

In [ ]:
orders_test = orders[orders.eval_set=='test']
orders_prior = orders[orders.eval_set=='prior']
orders_train = orders[orders.eval_set=='train']

The aisle and department are all we really care about. These are used to add more information
to the individual items ordered. Product name is not important. Well it could be if start to
factor in words in the name, like coookie or organic. That is beyond the scope of this
notebook.

In [ ]:
orders_train.head()


In [ ]:
orders_prior[orders_prior.user_id ==1]

In [ ]:
display(order_prior[order_prior.order_id==2539329])
display(order_prior[order_prior.order_id==2398795])
display(order_prior[order_prior.order_id==473747])
display(order_prior[order_prior.order_id==2254736])
display(order_prior[order_prior.order_id==431534])
display(order_prior[order_prior.order_id==3367565])
display(order_prior[order_prior.order_id==550135])
display(order_prior[order_prior.order_id==3108588])
display(order_prior[order_prior.order_id==2295261])
display(order_prior[order_prior.order_id==2550362])


In [ ]:
import csv
start = time.time()
f_products = dict()
with open("data/products.csv",  "r", encoding='utf8') as f_obj:
    reader = csv.reader(f_obj)
    print (next(reader))
    for row in reader:
        product_id = int(row[0])
        x = [int(row[2]), int(row[3])]
        f_products[product_id] = x
end = time.time()
print('{0} seconds elapsed time '.format(end - start))
print (len(f_products))

In [ ]:
products.head(5)

In [ ]:
for i in range(5):
    print (f_products[i+1])

In [ ]:
def f_order_order_id(x):
    return x[0]
def f_order_product_id(x):
    return x[1]
def f_order_aisle_id(x):
    return x[2]
def f_order_department_id(x):
    return x[3]
def f_order_reordered(x):
    return x[4]

We dont care about which order in the cart it was. It's possible that aisle and department matter so we add those in. Maybe the person tends to order more from certain departments.

In [ ]:
import csv
start = time.time()
f_order_train = list()
with open("data/order_products__train.csv", "r", encoding='utf8') as f_obj:
    reader = csv.reader(f_obj)
    print (next(reader))
    for row in reader:
        reordered = int(row[3])
        if (reordered == 0):
            continue
        product_id = int(row[1])
        product_loc = f_products[product_id] 
        x = [int(row[0]), int(row[1]), product_loc[0],
             product_loc[1], int(row[3])]
        f_order_train.append(x)
end = time.time()
print('{0} seconds elapsed time '.format(end - start))
print ('{0} order_train records'.format(len(f_order_train)))

In [ ]:
for ix in range(5):
    x = f_order_train[ix]
    print (x, f_order_order_id(x), f_order_product_id(x), f_order_reordered(x))
    display (products[products.product_id == f_order_product_id(x)])

In [ ]:
order_train.head(5)

In [ ]:
import csv
start = time.time()
f_order_prior = list()
with open("data/order_products__prior.csv", "r", encoding='utf8') as f_obj:
    reader = csv.reader(f_obj)
    print (next(reader))
    for row in reader:
        product_id = int(row[1])
        reordered = int(row[3])
        if (reordered == 0):
            continue
        product_loc = f_products[product_id] 
        x = [int(row[0]), int(row[1]), product_loc[0],
             product_loc[1], int(row[3])]

        f_order_prior.append(x)
end = time.time()
print('{0} seconds elapsed time '.format(end - start))
print ('{0} order_train records'.format(len(f_order_prior)))

In [ ]:
for ix in range(5):
    x = f_order_prior[ix]
    print (x, f_order_order_id(x), f_order_product_id(x), f_order_reordered(x))
    display (products[products.product_id == f_order_product_id(x)])

The meaning of fields in aisles are self explanatory.

In [ ]:
order_prior.head(5)

The meaning of fields in departments are self explanatory.

The meaning of fields in products are self explanatory.

Order_train and order_prior have a slightly non intuitive structure. Each order has a set of 
products. For each product we are also given which order it appears in the cart as well as if 
that customer has ordered the product before. The order which it appears may indicate how they went through the aisles. These are essentially descriptions of a cart. Prior seems to be orders prior to those in the supplied training set. In other words, those are being used to to predict order_train.

For this solution, add_to_cart_order is not needed and so it's better to just get rid of it here to avoid complications later

The order carts have been divided into two tables, the order_train and order_prio.

In [ ]:
def orders_order_id(x):
    return x[0]
def orders_user_id(x):
    return x[1]
def orders_eval_set(x):
    return x[2]

In [ ]:
# we ignore records with no days_since_prior_order for now
# it might make sense to make it equal to the one more than the largest
# one found so tha we can normalize the numbers and not have that one be like
# some outlier anomoly
#
import csv
start = time.time()
f_orders = list()
f_prior_orders = list()
f_train_orders = list()
f_test_orders = list()
with open("data/orders.csv", "r", encoding='utf8') as f_obj:
    reader = csv.reader(f_obj)
    print (next(reader))
    for row in reader:
        if (row[6] == ''):
            None # no value so we ignore it.
        else:
            x = [int(row[0]), int(row[1]), row[2], int(row[3]), int(row[4]), int(row[5]),
                 float(row[6])]
            f_orders.append(x)
            eval_set = row[2]
            if eval_set == 'train':
                f_train_orders.append(x)
            elif eval_set == "prior":
                f_prior_orders.append(x)
            elif eval_set == "test":
                f_test_orders.append(x)
            else:
                print("SOMETHING WRONG READING ORDERS.CSV")
end = time.time()
print('{0} seconds elapsed time '.format(end - start))
print (len(f_orders))
print (len(f_prior_orders))
print (len(f_train_orders))
print (len(f_test_orders))
print (len(f_orders), ' = ' , len(f_prior_orders) + len(f_train_orders) + len(f_test_orders))

In [ ]:
for ix in range(5):
    x = f_orders[ix]
    print (x)

In [ ]:

display (orders.head(5))
print (len(orders))
print (orders.dtypes)

Nan for days since prior order means there have been no prior orders for that customer. This will be the case when order_number==1.

There is a general probability that a product will be reordered if it has already been ordered. For each product, you can estimate the probability of it being reordered across the population.

During development, to test the algorithm we train with prior and test
with train. During production, we train with prior and train. This way during model development we will have a set of answers to use.


In [ ]:
def makeUserIds():
    user_ids = set()
    for x in f_orders:
        user_ids.add(orders_user_id(x))
    return user_ids

In [ ]:
user_ids = makeUserIds()

In [ ]:
len(user_ids)

In [ ]:
def initProductsOfUser():
    productsOfUser = dict()
    for i in user_ids:
        productsOfUser[i] = set()
    return productsOfUser

In [ ]:
productsOfUser = initProductsOfUser()

In [ ]:
len(productsOfUser)

We separately keep a list of reordered products by user, and the number of times reordered


In [ ]:
def initReorderedProductsOfUser():
    reorderedProductsOfUser = dict()
    for i in user_ids:
        reorderedProductsOfUser[i] = dict()
    return reorderedProductsOfUser

In [ ]:
reorderedProductsOfUser = initReorderedProductsOfUser()

In [ ]:
len(reorderedProductsOfUser)

# Good place to start from

When we are in development mode, then we use the training set to develop the algorithm. When we move to production mode, use the entire train+prior orders to develop the model. Possibly we should always train with the training set. TBD.....

In [ ]:
development = False
if development:
    f_test_orders = f_train_orders
    f_orders = f_prior_orders
    f_test_order = f_order_train
    
    test_orders = train_orders
    orders = prior_orders
    test_order = order_train
    display(test_orders.head(5))
    display(test_order.head(5))

we calculate the number of orders from each user. For now prior + training

In [ ]:
num_times_ordered = dict()
for order in f_orders:
    user_id = orders_user_id(order)
    if user_id not in num_times_ordered:
        num_times_ordered[user_id] = 1
    else:
        num_times_ordered[user_id] = num_times_ordered[user_id] + 1
     

In [ ]:
num_times_ordered_prior = dict()
for order in f_orders:
    user_id = orders_user_id(order)
    eval_set = orders_eval_set(order)
    if eval_set != 'prior':
        continue
    if user_id not in num_times_ordered_prior:
        num_times_ordered_prior[user_id] = 1
    else:
        num_times_ordered_prior[user_id] = num_times_ordered_prior[user_id] + 1



In [ ]:
len(num_times_ordered_prior)
print(num_times_ordered_prior[1], num_times_ordered[1])


We only do non test orders

In [ ]:
len(num_times_ordered)

In [ ]:
def f_makeOrdersOfUser():
    orders_of_user = dict()
    user_of_orders = dict()
    orders_info = dict()
    for u in user_ids:
        orders_of_user[u] = list()
    for x in f_orders:
        #display(row)
        if x[2] != 'test':
            x_ = [x[0], x[1], x[3], x[4], x[5], x[6]]
            user_id = orders_user_id(x)
            order_id = orders_order_id(x)
            user_of_orders[order_id] = user_id
            orders_of_user[user_id].append(x_)
            orders_info[order_id] = x_
    return orders_of_user, user_of_orders, orders_info

In [ ]:
start = time.time()
orders_of_user, user_of_orders, orders_info = f_makeOrdersOfUser()
end = time.time()
print('{0} seconds elapsed time '.format(end - start))
print(orders_info[157550])

In [ ]:
print(len(orders_of_user), len(user_of_orders))
orders_of_user[20]

For now, we train with the whole dataaset and use the robots for test.


In [ ]:
order_group = dict()
f_order_prior[1]

In [ ]:
productProbabilityOfUser = dict()
for u in user_ids:
    productProbabilityOfUser[u] = dict()

In [ ]:
productProbabilityOfUserPrior = dict()
for u in user_ids:
    productProbabilityOfUserPrior[u] = dict()

In [ ]:
def makeProductsOfUser():
    num_skipped = 0
    for x in f_order_prior:
        #display(row)
        #print(x)
        order_id = f_order_order_id(x)
        if order_id in order_group:
            order_group[order_id].append(x)
        else:
            order_group[order_id] = [x]
        if not order_id in user_of_orders:
            num_skipped = num_skipped + 1
            continue
        user_id = user_of_orders[order_id]
        productsOfUser[user_id].add(f_order_product_id(x))
    for x in f_order_train:
        #display(row)
        order_id = f_order_order_id(x)
        if order_id in order_group:
            order_group[order_id].append(x)
        else:
            order_group[order_id] = [x]
        if not order_id in user_of_orders:
            num_skipped = num_skipped + 1
            continue
        user_id = user_of_orders[order_id]
        productsOfUser[user_id].add(f_order_product_id(x))
    print("num skiped: ", num_skipped)

In [ ]:
def makeProductsProbabilityOfUser():
    num_skipped = 0
    for x in f_order_prior:
        #display(row)
        #print(x)
        order_id = f_order_order_id(x)
        user_id = user_of_orders[order_id]
        product_id = f_order_product_id(x)
        num_times = num_times_ordered[user_id]
        num_times_inv = 1.0 / num_times
        if product_id in productProbabilityOfUser[user_id]:
            productProbabilityOfUser[user_id][product_id] = productProbabilityOfUser[user_id][product_id] + num_times_inv
        else:
            productProbabilityOfUser[user_id][product_id] = num_times_inv
    for x in f_order_train:
        #display(row)
        #print(x)
        order_id = f_order_order_id(x)
        user_id = user_of_orders[order_id]
        product_id = f_order_product_id(x)
        num_times = num_times_ordered[user_id]
        num_times_inv = 1.0 / num_times
        if product_id in productProbabilityOfUser[user_id]:
            productProbabilityOfUser[user_id][product_id] = productProbabilityOfUser[user_id][product_id] + num_times_inv
        else:
            productProbabilityOfUser[user_id][product_id] = num_times_inv

In [ ]:
def makeProductsProbabilityOfUserPrior():
    num_skipped = 0
    for x in f_order_prior:
        #display(row)
        #print(x)
        order_id = f_order_order_id(x)
        user_id = user_of_orders[order_id]
        product_id = f_order_product_id(x)
        num_times = num_times_ordered_prior[user_id]
        num_times_inv = 1.0 / num_times
        if product_id in productProbabilityOfUserPrior[user_id]:
            productProbabilityOfUserPrior[user_id][product_id] = productProbabilityOfUserPrior[user_id][product_id] + num_times_inv
        else:
            productProbabilityOfUserPrior[user_id][product_id] = num_times_inv
 

In [ ]:
start = time.time()
makeProductsOfUser()
end = time.time()
print(end - start)

In [ ]:
start = time.time()
makeProductsProbabilityOfUser()
end = time.time()
print(end - start)

In [ ]:
productProbabilityOfUser[1]

In [ ]:
start = time.time()
makeProductsProbabilityOfUserPrior()
end = time.time()
print(end - start)

productProbabilityOfUserPrior[1]

In [ ]:
len(productsOfUser)

f_order_prior[12]


In [ ]:
order_group[2398795]

In [ ]:
def f_allOrdersOfUser(user_id):
    return orders_of_user[user_id]

In [ ]:
def f_allOrderOfOrdersReordered(order_id):
    result = list()
    if order_id not in order_group:
        return result
    for x in order_group[order_id]:
        if x[len(x)-1] == 1:
            result.append(x)
    return result

In [ ]:
f_allOrderOfOrdersReordered(2398795)

Figure out avergage number of orders per user

In [ ]:
round(0.51)

In [ ]:
average_num_reorders_per = dict()
for u in user_ids:
    num_orders = 0.0;
    num_products_ordered = 0.0;
    for order in f_allOrdersOfUser(u):
        num_orders = num_orders + 1
        for prec in f_allOrderOfOrdersReordered(order[0]):
            num_products_ordered = num_products_ordered + 1
    if num_orders > 0:
        average = round(num_products_ordered / num_orders)
    else:
        average = 0
    average_num_reorders_per[u] = average
    print (average)

In [ ]:
def f_productIdList(ogroup):
    result = list()
    for o in ogroup:
        result.append(o[1])
    return result

In [ ]:
def f_makeOneOrder(order_id, product_id, reordered):
    ploc = f_products[product_id]
    return [order_id, product_id, ploc[0], ploc[1], reordered]

In [ ]:
f_makeOneOrder(22,23909, 1)

In [ ]:
def f_makeMultipleOrder(order_id, product_id_list, reordered):
    result = list()
    for o in product_id_list:
       result.append(f_makeOneOrder(order_id, o, reordered))
    return result

In [ ]:
f_makeMultipleOrder(22, [23909, 22754], 0)

In [ ]:
def allProductsUser(user_id):
    return productsOfUser[user_id]

In [ ]:
allProductsUser(1)

We need to make all the order records now. This means making ones that use the other products that this person orders but we know that he did not reorder them at that time.
For each order of a user, we need to see what products had a repeat order and then add a no repeat order for all other products that they have ordered.

Can add another column ordered so we can record if the product was ordered at all as opposed to both not ordered and not reordered

For each of orders, we need data for all the other things this use might reorder.

We probably need another decision net for products that have never been reordered.

For now we are using the whole data set for training.

We do this all one user at a time for now.


In [ ]:
enable_randomList = True

In [ ]:
#We want to be able to chose maxNum items from a list of products
# if we have less than maxNum items then can return the original list
# ditto for =0 which is just a special case to do nothing and return
# the original list
#
import random 
def randomList(l, maxNum ):
    return set(l)
    l_ = list()
    #print("l: ", l)
    if enable_randomList & (len(l) > 0) & (maxNum != 0) & (maxNum < len(l)):
        # a is guaranteed to be between 0 and 1
         a = maxNum / len(l)  
         for i in l:
             if random.random() < a:
                 l_.append(i)
         l = l_
    #print("l_: " , l_)                     
    lSet = set(l)
    return lSet
                         
# we go through all the orders or a user, and look at the products
# reordered. we know that at that time, all other products that the person
# has ordered were not reordered. We could use that whole list, but it
# can be large so we randomly select a subset of that which has the same
# cardinality of what was ordered
#


In [ ]:
def f_makeNonReorder(user_id, maxNum = 0):
    allP = list(allProductsUser(user_id))
    # for each order set for the user, we go through the individal orders
    # and add all the negatives for any not there as recurring.
    #
    x = f_allOrdersOfUser(user_id)
    result = list()
    for o in x:
        oo = f_allOrderOfOrdersReordered(o[0])
        p = f_productIdList(oo)
        allPSet = set(randomList(allP, len(p)))
        butP = allPSet.difference(set(p))
        result.extend(f_makeMultipleOrder(o[0], butP, 0))
    return result

In [ ]:
x = f_makeNonReorder(1)


In [ ]:
def f_makeFullOrder(user_id):
    x  = f_allOrdersOfUser(user_id)
    result = list()
    for o in x:
        #print(o, o[0])
        result.extend(f_allOrderOfOrdersReordered(o[0]))
    result.extend(f_makeNonReorder(user_id))
    return result

In [ ]:
x = f_makeFullOrder(1)
len(x)



In [ ]:
train_accuracy = list()
train_accuracy_test = list()
train_f1 = list()
train_f1_test = list()
train_accuracy_mytest = list()
train_f1_mytest = list()
train_accuracy_blended = list()
train_f1_blended = list()

In [ ]:
def resetStats():
    train_accuracy = list()
    train_accuracy_test = list()
    train_f1 = list()
    train_f1_test = list()
    train_accuracy_mytest = list()
    train_f1_mytest = list()
    train_accuracy_blended = list()
    train_f1_blended = list()

In [ ]:
def predictByUser(user_id, product_id):
    if product_id in productProbabilityOfUser[user_id] :
        if productProbabilityOfUser[user_id][product_id] >= 0.50:
             return 1
    return 0

In [ ]:
def predictByUserPrior(user_id, product_id):
    if product_id in productProbabilityOfUserPrior[user_id] :
        if productProbabilityOfUserPrior[user_id][product_id] >= 0.50:
             return 1
    return 0

In [ ]:
# we want to add a negative for each item in the test_list that has a positive,
# for those that don't already have one
def finishTestList(user_id, x, y, ypred):
    if len(x) == 0:
        return x,y
    user_prods = allProductsUser(user_id)
    prods = set()
    newY = y
    newYPred = ypred
    newX = x
    tmpl = x[0]
    #print("tmpl: ", tmpl)
    for r in x:
        #print(r[0], int(r[0]))
        prods.add(r[0])
    for p in user_prods:
        if not p in prods:
            ploc = f_products[p]
            tmpl_ = list(tmpl)
            tmpl_[0] = p# float(p)
            tmpl_[2] = ploc[0]#float(ploc[0])
            tmpl_[3] = ploc[1]#float(ploc[1])
            #print("tmpl_: ", tmpl_)
            newX.append(tmpl_)
            newY.append(0)
            newYPred.append(predictByUserPrior(user_id, p))
    #print("final newX: ", newX)
    return newX, newY, newYPred

order_id	user_id	eval_set	order_number	order_dow	order_hour_of_day	days_since_prior_order

In [ ]:
def newPredict(newOrder): 
    user_id = newOrder[1]
    order_num = newOrder[3]
    order_dow = newOrder[4]    
    order_hour_of_day = newOrder[5]
    days_since_prior = newOrder[6]
    num_orders = num_times_ordered[user_id]
    plist = list()
    #print("new order: ", newOrder)
    # compute average numbers of orders for a user
    
    for p in allProductsUser(user_id):
        order_nums = list()
        #print("one product: ", p)
        num_p = 0.0
        num_p_dow = 0.0
        num_p_hour = 0.0
        num_p_since = 0.0
        #num_p_prev = 0.0
        for order in f_allOrdersOfUser(user_id):
            #print("one master order")
            order_id_ = order[0]
            order_num_ = order[2]
            order_dow_ = order[3]    
            order_hour_of_day_ = order[4]
            days_since_prior_ = order[5]
            #print(order, this_days_since_prior_order)
            for o in f_allOrderOfOrdersReordered(order_id_):
                p_ = o[1]
                if (p == p_):
                    #print ('match: ', p_)
                    num_p = num_p + 1
                    order_nums.append(order_num_)
                    #if (order_num == order_num_+1):
                    #    num_p_prev = num_p_prev + 1
                    if order_dow == order_dow_:
                        num_p_dow = num_p_dow + 1
                    if order_hour_of_day == order_hour_of_day_:
                        num_p_hour = num_p_hour + 1
                    if (days_since_prior == days_since_prior_) | \
                    (days_since_prior -1 == days_since_prior_) | \
                    (days_since_prior + 1 == days_since_prior_):
                            num_p_since = num_p_since + 1
        #print(p, num_orders, num_p, num_p_dow, num_p_hour, num_p_since)
        #print(order_nums)
        thresh = num_orders * 0.125 # probability
        if (num_p >= thresh) | (num_p_dow >= 2) | (num_p_hour >= 2) | (num_p_since >= 2):
            #print("add it", p)
            plist.append(p)
        else:
            if order_num - 1 in order_nums:
                #print("was ordered last time")
                num_conseq = 0
                for i in order_nums:
                    if i - 1 in order_nums:
                        num_conseq = num_conseq + 1
                #print('num conseq: ', num_conseq)
                if num_conseq >= 2:
                    #print("add it: ", p)
                    plist.append(p)
            if order_num - 2 in order_nums:
                #print("was ordered 2 times ago")
                num_conseq = 0
                for i in order_nums:
                    if i - 2 in order_nums:
                        num_conseq = num_conseq + 1
                #print('num conseq2: ', num_conseq)
                if num_conseq >= 2:
                    #print("add it: ", p)
                    plist.append(p)
              
            None #print("dont add it")
    return plist

In [ ]:
#rint(f_test_orders[0])
newPredict(f_test_orders[0])

In [ ]:
from sklearn.preprocessing import StandardScaler  
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import load_iris
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn import preprocessing
from sklearn.naive_bayes import GaussianNB

# compute the number of times this item has been reordered at the time
# of this order_num
#
def computeProdOrder(user_id):
    result = dict()
    pcount = dict()
    pairs = list()
    for o in f_allOrdersOfUser(user_id):
        order_num = o[2]
        #print(order_num)
        for ox in f_allOrderOfOrdersReordered(o[0]):
            product_id = ox[1]
            if product_id not in pcount:
                pcount[product_id] = 0
            #print(order_num, product_id)
            pairs.append([product_id, order_num])
    pairs.sort()
    #print(pcount)
    for p in pairs:
        product_id = p[0]
        order_num = p[1]
        pcount[product_id] = pcount[product_id] + 1
        if not product_id in result:
            result[product_id] = dict()
        result[product_id][order_num] = pcount[product_id]
    #print (result)
    return result

def numTimesReordered(d, p, o):
     if o not in d[p]: 
         #print ('num times: ', o)
         #print (d[p])
         klist = d[p].keys()
         #klist.sort()
         result = 0
         for k in klist:
             if o > k:
                return d[p][k]
         return 0
     else:
        return d[p][o]
    
def f_trainUser(user_id):
    prodOrderNum = computeProdOrder(user_id)
    allZ = f_makeFullOrder(user_id)
    xall = list()
    yall = list()
    # find max user_number and number of orders
    max_order_num = 0
    min_order_num = 1000000  # should use infinity
    for z in allZ:
        t = orders_info[z[0]][2]
        if t > max_order_num:
            max_order_num = t
        if t < min_order_num:
            min_order_num = t
    #print("min/ax order num: ", min_order_num, max_order_num)
    y = list()
    y_test = list()
    x = list()
    x_test = list()
    y_test_mypredict = list()
    #print(allZ[0])
    # for now just use test with last order num orders
    for z in allZ:
        #print(orders_info[z[0]])
        orderNum = orders_info[z[0]][2]
        product_id = z[1]
        #print(product_id, orderNum)
        oneY = z[-1]
        #yall.append(z[-1])
        #print(orders_info[z[0]])
        pct = (1.0 * numTimesReordered(prodOrderNum,product_id, orderNum))/ orderNum
        #print(pct, orderNum)
        oneX = z[1:-1]+ [pct] + orders_info[z[0]][2:]
        #oneX = z[1:]].append(orders_info[z[0]][2:])
        #print(orders_info[z[0]][2:])
        #oneX = list(map(float, oneX))
        product_id = oneX[0]
        if orderNum == max_order_num:
            x_test.append(oneX)
            y_test.append(oneY)
            y_test_mypredict.append(predictByUserPrior(user_id, product_id))
        else:
            x.append(oneX)
            y.append(oneY)
    #print(x)
    #print(x_test)
    #print("x: " , x)
    x_test, y_test, y_test_predict = finishTestList(user_id, x_test, y_test, y_test_mypredict)
    from sklearn.svm import SVC
    from sklearn import linear_model
    #clf = linear_model.LogisticRegression(C=1e5)
    #clf = SVC()    
    #clf = DecisionTreeClassifier(random_state=0)
    #clf = RandomForestClassifier(max_depth=2, random_state=0)
    #lstart = time.time()
    clf = MLPClassifier(solver='lbfgs', max_iter=200, 
                      hidden_layer_sizes=(32, 32), random_state=1)
    #clf = GaussianNB()
    #print ('template: ', x[0])
    if (len(x) ==0):
        return None, None, None, None
    #print("x[0]: ", x[0])
    #print("an x: ", x)
    from sklearn.preprocessing import OneHotEncoder
    #print(x)
    #print(x_test)
    enc = OneHotEncoder(categorical_features = [0, 1, 2, 5, 6]) 
    #print(enc)
    #print (x + x_test)
    enc.fit(x + x_test)
    #print(x[0])
    x_tmpl = list(x[0])
    #print('x_tmpl: ', x_tmpl)
    tpl = list()
    for xti, xt in enumerate(x_test):
        tpl.append(x_test[xti])
    x = enc.transform(x).toarray()
    #print(x)
    #print(type(x))
    x_test = enc.transform(x_test).toarray()
    for xti, xt in enumerate(x_test):
        tpl.append(xt)
    #print(x.shape, x_test.shape)
    #print(x[0])
    clf.fit(x, y)
    #lend = time.time()
    #print('{0} training seconds elapsed time '.format(lend - lstart))
    #print (x)
    #print (y)
    #print(clf.score(x,y))
    if True | development:
        i = clf.predict(x)
        i_test = clf.predict(x_test)
        iprob_test = clf.predict_proba(x_test)
        #for xti, xt in enumerate(x_test):
        #    print('[', int(xt[0]), i_test[xti], iprob_test[xti], ']', end='')
        #    print();

    #print(i)
    from sklearn.metrics import accuracy_score
    from sklearn.metrics import f1_score
    if True | development :
        accuracy = accuracy_score(y, i)
        train_accuracy.append(accuracy)
        f1 = f1_score(y, i)
        train_f1.append(f1)
        accuracy_test = accuracy_score(y_test, i_test)
        train_accuracy_test.append(accuracy_test)
        f1_test =  f1_score(y_test, i_test)
        train_f1_test.append(f1_test)
        accuracy_mytest = accuracy_score(y_test, y_test_mypredict)
        train_accuracy_mytest.append(accuracy_mytest)
        f1_mytest =  f1_score(y_test, y_test_mypredict)
        train_f1_mytest.append(f1_mytest)
        if f1_mytest > f1_test:
            f1_blended = f1_mytest
            accuracy_blended = accuracy_mytest
        else:
            f1_blended = f1_test
            accuracy_blended = accuracy_test
        train_accuracy_blended.append(accuracy_blended)
        train_f1_blended.append(f1_blended)
        #print('test diff for user: ', user_id)
        #for xt in x_test:
        #    print (xt)
        y_prod = list()
        y_prod_test = list()
        y_prod_mytest = list()
        #print(y_test)
        for yti, yt in enumerate(y_test):
            if (yt == 1): y_prod.append(int(tpl[yti][0]))
        for yti, yt in enumerate(i_test):
            if (yt == 1): y_prod_test.append(int(tpl[yti][0]))
        for yti, yt in enumerate(y_test_mypredict):
            if (yt == 1): y_prod_mytest.append(int(tpl[yti][0]))
        y_prod.sort()
        y_prod_test.sort()
        y_prod_mytest.sort()
        print(y_prod)
        print(y_prod_test)
        print(y_prod_mytest)
        print(user_id, "train accuracy: " , accuracy_score(y, i), " f1: ", f1_score(y, i))
        print(user_id, "test accuracy: " , accuracy_score(y_test, i_test), " f1: ", f1_score(y_test, i_test))
        print(user_id, "mytest accuracy: " , accuracy_score(y_test, y_test_mypredict), 
              " f1: ", f1_score(y_test, y_test_mypredict))
        #print('num samples train: ', len(x))
        #print('num samples test: ', len(x_test))
    if f1_mytest >= (f1_test * 1.0):
        return None, None, None, None
    else:
        return len(x), clf, x_tmpl, enc # scaler

In [ ]:

f_trainUser(35)
f_trainUser(4)
#f_trainUser(35)
#f_trainUser(50)
print(np.mean(train_accuracy_test))

In [ ]:
from sklearn.preprocessing import StandardScaler  
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import load_iris
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn import preprocessing
from sklearn.naive_bayes import GaussianNB

# compute the number of times this item has been reordered at the time
# of this order_num
#
     
    
def f_new_trainUser(user_id):
    prodOrderNum = computeProdOrder(user_id)
    allZ = f_makeFullOrder(user_id)
    xall = list()
    yall = list()
    # find max user_number and number of orders
    max_order_num = 0
    min_order_num = 1000000  # should use infinity
    for z in allZ:
        t = orders_info[z[0]][2]
        if t > max_order_num:
            max_order_num = t
        if t < min_order_num:
            min_order_num = t
    #print("min/ax order num: ", min_order_num, max_order_num)
    y = list()
    y_test = list()
    x = list()
    x_test = list()
    y_test_mypredict = list()
    #print(allZ[0])
    # for now just use test with last order num orders
    for z in allZ:
        #print(orders_info[z[0]])
        orderNum = orders_info[z[0]][2]
        product_id = z[1]
        #print(product_id, orderNum)
        oneY = z[-1]
        #yall.append(z[-1])
        #print(orders_info[z[0]])
        pct = (1.0 * numTimesReordered(prodOrderNum,product_id, orderNum))/ orderNum
        #print(pct, orderNum)
        oneX = z[1:-1]+ [pct] + orders_info[z[0]][2:]
        #oneX = z[1:]].append(orders_info[z[0]][2:])
        #print(orders_info[z[0]][2:])
        #oneX = list(map(float, oneX))
        product_id = oneX[0]
        if orderNum == max_order_num:
            x_test.append(oneX)
            y_test.append(oneY)
            y_test_mypredict.append(predictByUserPrior(user_id, product_id))
        else:
            x.append(oneX)
            y.append(oneY)
    #print(x)
    #print(x_test)
    #print("x: " , x)
    x_test, y_test, y_test_predict = finishTestList(user_id, x_test, y_test, y_test_mypredict)
    from sklearn.svm import SVC
    from sklearn import linear_model
    #clf = linear_model.LogisticRegression(C=1e5)
    #clf = SVC()    
    #clf = DecisionTreeClassifier(random_state=0)
    #clf = RandomForestClassifier(max_depth=2, random_state=0)
    #lstart = time.time()
    clf = MLPClassifier(solver='lbfgs', max_iter=200, 
                      hidden_layer_sizes=(32, 32), random_state=1)
    #clf = GaussianNB()
    #print ('template: ', x[0])
    if (len(x) ==0):
        return None, None, None, None
    #print("x[0]: ", x[0])
    #print("an x: ", x)
    from sklearn.preprocessing import OneHotEncoder
    #print(x)
    #print(x_test)
    enc = OneHotEncoder(categorical_features = [0, 1, 2, 5, 6]) 
    #print(enc)
    #print (x + x_test)
    enc.fit(x + x_test)
    #print(x[0])
    x_tmpl = list(x[0])
    #print('x_tmpl: ', x_tmpl)
    tpl = list()
    for xti, xt in enumerate(x_test):
        tpl.append(x_test[xti])
    x = enc.transform(x).toarray()
    #print(x)
    #print(type(x))
    x_test = enc.transform(x_test).toarray()
    for xti, xt in enumerate(x_test):
        tpl.append(xt)
    #print(x.shape, x_test.shape)
    #print(x[0])
    clf.fit(x, y)
    #lend = time.time()
    #print('{0} training seconds elapsed time '.format(lend - lstart))
    #print (x)
    #print (y)
    #print(clf.score(x,y))
    if True | development:
        i = clf.predict(x)
        i_test = clf.predict(x_test)
        iprob_test = clf.predict_proba(x_test)
        #for xti, xt in enumerate(x_test):
        #    print('[', int(xt[0]), i_test[xti], iprob_test[xti], ']', end='')
        #    print();

    #print(i)
    from sklearn.metrics import accuracy_score
    from sklearn.metrics import f1_score
    if True | development :
        accuracy = accuracy_score(y, i)
        train_accuracy.append(accuracy)
        f1 = f1_score(y, i)
        train_f1.append(f1)
        accuracy_test = accuracy_score(y_test, i_test)
        train_accuracy_test.append(accuracy_test)
        f1_test =  f1_score(y_test, i_test)
        train_f1_test.append(f1_test)
        accuracy_mytest = accuracy_score(y_test, y_test_mypredict)
        train_accuracy_mytest.append(accuracy_mytest)
        f1_mytest =  f1_score(y_test, y_test_mypredict)
        train_f1_mytest.append(f1_mytest)
        if f1_mytest > f1_test:
            f1_blended = f1_mytest
            accuracy_blended = accuracy_mytest
        else:
            f1_blended = f1_test
            accuracy_blended = accuracy_test
        train_accuracy_blended.append(accuracy_blended)
        train_f1_blended.append(f1_blended)
        #print('test diff for user: ', user_id)
        #for xt in x_test:
        #    print (xt)
        y_prod = list()
        y_prod_test = list()
        y_prod_mytest = list()
        #print(y_test)
        for yti, yt in enumerate(y_test):
            if (yt == 1): y_prod.append(int(tpl[yti][0]))
        for yti, yt in enumerate(i_test):
            if (yt == 1): y_prod_test.append(int(tpl[yti][0]))
        for yti, yt in enumerate(y_test_mypredict):
            if (yt == 1): y_prod_mytest.append(int(tpl[yti][0]))
        y_prod.sort()
        y_prod_test.sort()
        y_prod_mytest.sort()
        print(y_prod)
        print(y_prod_test)
        print(y_prod_mytest)
        print(user_id, "train accuracy: " , accuracy_score(y, i), " f1: ", f1_score(y, i))
        print(user_id, "test accuracy: " , accuracy_score(y_test, i_test), " f1: ", f1_score(y_test, i_test))
        print(user_id, "mytest accuracy: " , accuracy_score(y_test, y_test_mypredict), 
              " f1: ", f1_score(y_test, y_test_mypredict))
        #print('num samples train: ', len(x))
        #print('num samples test: ', len(x_test))
    if f1_mytest >= (f1_test * 1.0):
        return None, None, None, None
    else:
        return len(x), clf, x_tmpl, enc # scaler

In [ ]:
def trainAllUser(f, row_first, num_rows, do_all=False):
    row_last = row_first + num_rows - 1
    row_num = -1
    t = test_orders.sort_values('order_id')
    correct_answer = 0.0
    total_answer = 0.0
    for index, row in t.iterrows():
        row_num = row_num + 1
        order_id = row['order_id']
        if (row_num < row_first) | (row_num > row_last):
            continue
        user_id = row['user_id']
        #print ('training user: ', user_id)
        x, clf, onepd = trainUser(user_id)
        #print(x)
        t1 = test_order[test_order.order_id == order_id]
        #display(row)
        #display(onepd)
        #display(t1)
        t1 = t1[t1.reordered==1]
        for index1, row1 in t1.iterrows():
            #print("row1")
            #display(row1)
            onepd['product_id'] = row1['product_id']
            #display(onepd)
            one = onepd.as_matrix().astype(float)
            #print(one)
            total_answer = total_answer + 1.0
            i = clf.predict([one])
            if i[0] == 1:
                correct_answer = correct_answer + 1.0
            #print(i)
            
        """
        p = allProductsUser(user_id)
        #print(p)
        #display(onepd)
        plist = list()
        for pi in p:
            onepd['product_id'] = pi
            one = onepd.as_matrix().astype(float)
            #print(one)
            i = clf.predict([one])
            if i[0] == 1:
                plist.append(pi)
        f.write('{0}, '.format(order_id))
        if plist == list():
            f.write('{0}'.format('None'))
        else:
            for i in plist:
                f.write('{0} '.format(i))
        f.write('\n')
        """
    print("percentage correct: ", correct_answer / total_answer)    

In [ ]:
def f_makeSubmission(f, row_first, num_rows, do_all=False):
    resetStats()
    row_last = row_first + num_rows - 1
    row_num = -1
    #time_for_submissions = 0.0
    #num_submissions = 0
    num_simple = 0
    num_not_simple = 0
    for x in f_test_orders:
        #stime_begin = time.time()
        row_num = row_num + 1
        if (row_num < row_first) | (row_num > row_last):
            if do_all:
               None
            continue
        order_id = x[0]
        user_id = x[1]
        p = allProductsUser(user_id)
        plist = list()
        if len(p) > 0:
            #ttime_begin = time.time()
            x, clf, x_tmpl, enc = f_trainUser(user_id)
            if clf == None:
                print('using simple predict')
                num_simple = num_simple + 1
                f.write('{0}, '.format(order_id))
                wrote_some=False
                for i in productProbabilityOfUser[user_id]:
                    if productProbabilityOfUser[user_id][i] >= 0.25:
                        f.write('{0} '.format(i))
                        wrote_some = True
                if not wrote_some:
                    f.write('None')
                f.write('\n')
                continue
            print('not using simple predict')
            num_not_simple = num_not_simple + 1
            #ttime_end = time.time()
            #print(" seconds to train user: ", ttime_end - ttime_begin)
            for pi in p:
                tmpl = list(x_tmpl)
                tmpl[0] = pi
                ploc = f_products[pi]
                tmpl[1] = ploc[0]
                tmpl[2] = ploc[1]
                tmpl = enc.transform(tmpl).toarray()
                #i = clf.predict(scaler.transform([tmpl]))
                i = clf.predict(tmpl)
                if i[0] == 1:
                    plist.append(pi)
                    #print("about to add: ", tmpl)
        f.write('{0}, '.format(order_id))
        if plist == list():
            f.write('{0}'.format('None'))
        else:
            for i in plist:
                f.write('{0} '.format(i))
        f.write('\n')
    print('train accuracy: ', np.mean(train_accuracy))
    print('train f1: ', np.mean(train_f1))
    print("test accuracy: ", np.mean(train_accuracy_test))
    print("test f1: ", np.mean(train_f1_test))
    print("mytest accuracy: ", np.mean(train_accuracy_mytest))
    print("mytest f1: ", np.mean(train_f1_mytest))
    print("blended accuracy: ", np.mean(train_accuracy_blended))
    print("blended f1: ", np.mean(train_f1_blended))
    print("percent simple: ", (num_simple * 1.0)/(num_simple + num_not_simple))
#        stime_end = time.time()
#       print(" seconds to do submission: ", stime_end - stime_begin)
#      num_submissions = num_submissions + 1
#        time_for_submissions = time_for_submissions + (stime_end - stime_begin)
#    print("average submission time: " , time_for_submissions / num_submissions)
#   print("total submissions time: ", time_for_submissions, num_submissions)

In [ ]:
import time
import sys
def makeAllSubmission() :
    with open('submissionall.csv', 'w') as f:
        f.write('order_id,products\n')
        element=0
        for t in f_test_orders:
            user_id = orders_user_id(t)
            order_id = orders_order_id(t)
             #print("order: ", order_id)
            p = productsOfUser[user_id]
            plist = list(p)
            f.write('{0}, '.format(order_id))
            for i in plist:
                f.write('{0} '.format(i))
            f.write('\n')




In [ ]:
import time
import sys
def makeAllHighPrioritySubmission() :
    with open('submissionhighall.csv', 'w') as f:
        f.write('order_id,products\n')
        element=0
        for t in f_test_orders:
            user_id = orders_user_id(t)
            order_id = orders_order_id(t)
             #print("order: ", order_id)
            f.write('{0}, '.format(order_id))
            wrote_some=False
            plist = list()
            for i in productProbabilityOfUser[user_id]:
                if productProbabilityOfUser[user_id][i] >= 0.250:
                    plist.append(i)
            plist.sort(reverse=True)
            average = average_num_reorders_per[user_id]
            num = 0
            for i in plist:
                if (num > average):
                    break
                f.write('{0} '.format(i))
                #print('{0} '.format(i))
                wrote_some = True
            if not wrote_some:
                f.write('None')
            f.write('\n')



In [ ]:
makeAllHighPrioritySubmission()

In [ ]:
import time
import sys
def makeAllNewSubmission() :
    with open('submissionnewall.csv', 'w') as f:
        f.write('order_id,products\n')
        element=0
        for t in f_test_orders:
            user_id = orders_user_id(t)
            order_id = orders_order_id(t)
            import time
            l = newPredict(t)
             #print("order: ", order_id)
            f.write('{0}, '.format(order_id))
            if len(l) != 0:
                for p in l:
                    f.write('{0} '.format(p))
                    wrote_some = True
            else:
                f.write('None')
            f.write('\n')




In [ ]:
makeAllNewSubmission()

In [ ]:
if not development:
    makeAllSubmission()

In [ ]:
def makeNullSubmission() :
    with open('submissionnull.csv', 'w') as f:
        f.write('order_id,products\n')
        t = test_orders.sort_values('order_id')
        for index, row in t.iterrows():
            f.write('{0}, {1}\n'.format(row['order_id'], 'None'))

In [ ]:
if not development:
    makeNullSubmission()

In [ ]:
print (enable_randomList)
enable_randomList = False
print (enable_randomList)

In [ ]:
import time
import sys
start = time.time()
if not development:
    f_makeSubmission(sys.stdout, 0, 100)
    
    #with open('submissionbig.csv', 'w') as f:
    #    f.write('order_id,products\n')
    #    f_makeSubmission(f, 0, 100000, True)
else:
    None #trainAllUser(sys.stdout, 0, 50)
end = time.time()
print(end - start)

In [ ]:
f_test_orders[0]

In [ ]:
2**100/2**9

In [ ]:
f_test_orders[1]

In [ ]:
f_train_orders[0]

In [ ]:
display(orders[orders.user_id==3])

In [ ]:
display(order_prior[order_prior.order_id==444309])

In [ ]:
f_products[35469]

In [ ]:
x = None
if x == None: print('yes')

In [ ]:
f_orders[2774568]


In [ ]:
f_orders

In [ ]:
f_test_orders